In [14]:
%matplotlib inline
import sys
print('Python version:', sys.version)

import numpy as np
print('Numpy version:', np.__version__)

import pandas as pd
print('Pandas version:', pd.__version__)

# Can be used for prelimianry data visualization, like mean, median, and mode
import matplotlib as mpl
import matplotlib.pyplot as plt
print('Matplotlib version:', mpl.__version__)

import datetime
import time

# Some constants to be used later
CONST_NUM_BOROUGHS = 5
max_turn_borough = 0

sns.set()
pal = sns.hls_palette(10, h=.5)
sns.set_palette(pal)

# Avoid display of scientific notation and show precision of 4 decimals:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Set option for showing all columns for cell display of a dataframe
pd.options.display.max_columns = None

Python version: 3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
Numpy version: 1.17.4
Pandas version: 0.25.3
Matplotlib version: 3.1.1
Seaborn version: 0.9.0


In [87]:
# Load complete dataset. No data dictionary is available.
# Source: https://data.cityofnewyork.us/Business/Legally-Operating-Businesses/w7w3-xahh
# Direct source: https://data.cityofnewyork.us/api/views/w7w3-xahh/rows.csv?accessType=DOWNLOAD&bom=true&format=true
df_business = pd.read_csv("./data/Legally_Operating_Businesses.csv")

# Source: https://data.cityofnewyork.us/Business/License-Applications/ptev-4hud
# Direct source: https://data.cityofnewyork.us/api/views/ptev-4hud/rows.csv?accessType=DOWNLOAD&bom=true&format=true
df_application = pd.read_csv("./data/License_Applications.csv")

# Additional dataset - contains direct mapping between zip code and borough in NYC
# Source: https://www.kaggle.com/kimjinyoung/nyc-borough-zip
# Direct source: https://www.kaggle.com/kimjinyoung/nyc-borough-zip/download
df_zip = pd.read_csv("./data/zip_borough.csv")



In [88]:
# Change objects to datetime format for easier calculation

# df_business
df_business['License Expiration Date'] = pd.to_datetime(df_business['License Expiration Date'])
df_business['License Creation Date'] = pd.to_datetime(df_business['License Creation Date'])

# df_application
df_application['Start Date'] = pd.to_datetime(df_application['Start Date'])
df_application['End Date'] = pd.to_datetime(df_application['End Date'])
df_application['Temp Op Letter Issued'] = pd.to_datetime(df_application['Temp Op Letter Issued'])
df_application['Temp Op Letter Expiration'] = pd.to_datetime(df_application['Temp Op Letter Expiration'])

# Convert zip codes to int (although actually float since some are NaN - Not a Number).
# However, Python can easily compare between flaot and integer: 2==2.0 return True
df_application['Zip'] = df_application['Zip'].apply(pd.to_numeric, errors='coerce')

# We merge two datasets to leverage the information of boroughs and zip codes
# Note the different column names in 2 datasets
df_join = pd.merge(df_application, df_zip, left_on='Zip', right_on='zip', how='inner', indicator=True)

# We also merge the two given datasets, to be used in Taks 4
# The most logical choice is to merge on License Number
df_merge = pd.merge(df_business, df_application, left_on='DCA License Number', right_on='License Number', how='inner',indicator=True)

In [22]:
# Preliminary data check

# print(df_business.head())
# print(df_business.tail())
# print(df_business.dtypes)
# df_business.info()

# ['individual', 'business']
# list(df_business['License Type'].str.lower().unique())

# ['active', 'inactive']
# list(df_business['License Status'].str.lower().unique())

# Around 30 different strings
# print(list(df_business['Industry'].str.lower().unique()))

# Too many cities
# list(df_business['Address City'].str.lower().unique())

# [nan, 'outside nyc', 'brooklyn', 'queens', 'bronx', 'manhattan', 'staten island']
# list(df_business['Address Borough'].str.lower().unique())

# [nan, 3.0, 4.0, 2.0, 1.0, 5.0]
# df_business['Borough Code'].unique()

# Around 50 different floats
# df_business['Community Board'].unique()

# Around 30 different floats
# df_business['Council District'].unique()


# print(df_application.head())
# print(df_application.tail())
# print(df_application.dtypes)
# df_application.info()


# print(df_zip.head())
# print(df_business.tail())
# print(df_business.dtypes)
# df_zip.info()


In [42]:
# ================ Task 1 ========================

# Converting df_zip to a dictionary for this task. 
# Later, we will also demonstrate how to join it with df_application
zip_dict = pd.Series(df_zip['borough'].values, index=df_zip['zip']).to_dict()

# A dictionary to contain turnaround time for each of the five boroughs
turnaround_dict = {}

# The for loop takes long in my local machine, since the data contains around 365 thousand rows (df_applications)
# The parameter of range should be changed to len(df_application)
for i in range(100):
    
# Now we extract the start and end dates for each application and calcualte the difference as turnaround
# Please note that some applications have endtime < starttime, which is maybe a potential glitch in data
    start = df_application.loc[i]['Start Date']
    end = df_application.loc[i]['End Date']
    
# Find the zip value for this business from the zip dataset
# This dataset is clean and hence guarantees accurate extraction of borough name against zip code
    zip_val = int(df_application.loc[i]['Zip'])
    
# Check if this zip is in NYC indeed. 
# Also the year is set to 2017    
    if start is not pd.NaT and end is not pd.NaT and zip_val in zip_dict and start.year==2017:
        borough_val = zip_dict[zip_val]
        
# Create a new list for this borough        
        if borough_val not in turnaround_dict.keys():
            turnaround_dict[borough_val] = list()
            
# Add the turnaround time for this business            
        turnaround_dict[borough_val].append(end - start)

# A list to keep the average turnaround value of all 5 boroughs 
avg_list = []
max_turn = 0

# We find the mean DCA license turnaround time
# At the same time, we also find the borough which boasts the highest turnaround time for businesses
for key in turnaround_dict.keys():

# We calculate the avg turnaround value for each borough
    avg_list.append(sum(turnaround_dict[key], datetime.timedelta(0))/len(turnaround_dict[key]))
    print("Avg for", key, "is =", avg_list[-1])

# We check if this is the highest so far. If yes, we mark it
    if len(avg_list)==1:
        max_turn = avg_list[0]
    if avg_list[-1] > max_turn:
        max_turn = avg_list[-1]
        max_turn_borough = key
        
# The list containing avg turnaround values is sorted
avg_list.sort(reverse=True)

Avg for Bronx is = 7 days 12:00:00
Avg for Brooklyn is = 3 days 13:20:00
Avg for Queens is = 1 days 20:34:17.142857
Avg for Manhattan is = 10 days 06:51:25.714285
Avg for Staten is = 3 days 00:00:00


In [41]:
# ===================== Task 2 =======================

# We use the different license/business category as keys to a dictionary
lic_category_set = set(df_join['License Category'].unique())
lic_category_dict = {x: [] for x in list(lic_category_set)}

# The parameter of range should be changed to len(df_join)
for i in range(4000):
    start = df_join.loc[i]['Start Date']
    end = df_join.loc[i]['End Date']
    
# Check if borough is our intended borough (with highest turnaround time). Also year is 2016.
    if df_join.loc[i]['borough'] == max_turn_borough and start.year==2016 and start is not pd.NaT and end is not pd.NaT:
        new_turn_val = end - start
        pos = df_join.loc[i]['License Category']
        lic_category_dict[pos].append(new_turn_val)

# We now have all turnaround values corresponding to all license categories
max_turn_in_borough = datetime.timedelta(0)
max_turn_category_in_borough = list(lic_category_dict.keys())[0]
avg_value = datetime.timedelta(0)

# Now need to find which license category boasts the highest value in this borough
for key in lic_category_dict.keys():
    if len(lic_category_dict[key]) == 1:
        avg_value = lic_category_dict[key][0]

    elif len(lic_category_dict[key]) > 1:
        avg_value = (sum(lic_category_dict[key], datetime.timedelta(0))/len(lic_category_dict[key]))
       
    if max_turn_in_borough < avg_value:
        max_turn_in_borough = avg_value
        max_turn_category_in_borough = key

print("max=", max_turn_in_borough, max_turn_category_in_borough)

max= 42 days 06:00:00 Sidewalk Cafe


In [80]:
# ==================== Task 3 =========================

# We have 2 very similar tasks - finding rate of application denial for 1) Application 2) Renewal Application
# Therefore, we write a method and call it with slightly different parameters

borough_set = set(df_join['borough'].unique())
borough_dict = {x: 0 for x in list(borough_set)}
total_dict_borough = {x: 0 for x in list(borough_set)}

print("Denial for new application:", find_app_denial_rate(df_join, borough_dict, total_dict_borough, 5000, 2018, 'Application'))
print("Denial for renewal:", find_app_denial_rate(df_join, borough_dict, total_dict_borough, 5000, 2018, 'Renewal'))

Denial for new application: ['Brooklyn', 0, 'Manhattan', 0.0625, 'Queens', 0, 'Bronx', 0.1564625850340136, 'Staten', 0]
Denial for renewal: ['Brooklyn', 0, 'Manhattan', 0.02666666666666667, 'Queens', 0, 'Bronx', 0.06666666666666667, 'Staten', 0]


In [78]:
def find_app_denial_rate(df_join, borough_dict, total_dict_borough, iterations, year, app_renew):
    for i in range(iterations):
        cur_borough = df_join.loc[i]['borough']
        start = df_join.loc[i]['Start Date']
        end = df_join.loc[i]['End Date']
        
        if start.year==year and df_join.loc[i]['Application or Renewal']==app_renew \
        and start is not pd.NaT and end is not pd.NaT:
            total_dict_borough[cur_borough] += 1
            if df_join.loc[i]['Status']=='Denied':
                borough_dict[cur_borough] += 1
    
    max_denial = 0
    max_denial_borough = ''
    for key,value in borough_dict.items():
        if value > max_denial:
            max_denial = value
            max_denial_borough = key
    denial_rate_list = []
    for (k,v), (k2,v2) in zip(borough_dict.items(), total_dict_borough.items()):
        denial_rate_list.append(k)
        denial_rate_list.append(v/v2 if v2 else 0) # Checking for division by zero

    return denial_rate_list

In [91]:
# ====================== Task 4 ==========================

# From the language of T4, it seems that the assignment has been created in 2018 and has not been updated.
# Therefore, there were 2 possibilities for me.
# 1. Follow T4 word-by-word. This would result in an empty file since no business with license expiring in 2018/2019 will have active status in the dataset
# 2. Change the date. Take the date as, say, Late 2020 and go by that.
# T4 does not specify that garage businesses have to be inside NYC

end_2020_datetime = pd.to_datetime('20201231', format='%Y%m%d', errors='coerce')
start_2020_datetime = pd.to_datetime('20200209', format='%Y%m%d', errors='coerce')

df_parking = pd.DataFrame()
count = 0

# The paramter should be len(df_merge)
for i in range(10000):
    if df_merge.loc[i]['License Status']=="Active" and df_merge.loc[i]['License Expiration Date']<=end_2020_datetime \
    and ("garage" in str(df_merge.loc[i]['License Category']).lower() or "parking" in str(df_merge.loc[i]['License Category']).lower()):
        count += 1
        df_parking = df_parking.append(df_merge.loc[i])
#         print(str(df_merge.loc[i]['License Category']).lower())
print(count)

# df_parking.drop(df_parking.columns.difference(['Start Date','End Date']), 1, inplace=True)

header = ["DCA License Number", "License Expiration Date", "License Status", "License Category", "End Date"]
df_parking.to_csv('parking.csv', columns = header)

8
